In [ ]:
import random
import pandas as pd
import re
import numpy as np
from random import randint

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/kubig_nlp프로젝트/10 31/final_script.csv")
df.drop('Unnamed: 0',axis=1, inplace=True)
df

,Q,A,role
0,정환아. 한 마디 해라.,"아, 진짜 이런 것 좀 하지 마세요.",정환
1,"♪ 실례, 실례하세요.",NaN,정환
2,♪ 맞아! 맞아! 맞-아! 어떻게 알고 왔죠.,NaN,정환
3,네. 근데 내일 바로 가신대요.,야!,정환
4,NaN,"손으로 드럽게, 씨..",정환
...,...,...,...
3708,NaN,우리 엄마 외롭지 않게 해주셔서 감사합니다 아저씨,선우
3709,NaN,쌍문동에서 오신 치타 사모님이 조용히 계시면 식이 진행되도록 하겠습니다.,동룡
3710,야 사회 내가 본다고 그랬잖아,사천에서 겨우 올라오 주제에 동룡이 잘 봐,택
3711,NaN,신랑 입장!,동룡


In [ ]:
df.isnull().sum()

Q       1111
A        145
role       0
dtype: int64

## 결측치 처리
QA 결측치가 너무 많다 : 해당 A와 유사도가 가장 비슷한 QA의 Q를 가져오자 (vice versa)

Q A 둘다 결측치인건 삭제

In [ ]:
df.drop(df[df['Q'].isnull() & df['A'].isnull()].index,inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Q,A,role
0,정환아. 한 마디 해라.,"아, 진짜 이런 것 좀 하지 마세요.",정환
1,"♪ 실례, 실례하세요.",NaN,정환
2,♪ 맞아! 맞아! 맞-아! 어떻게 알고 왔죠.,NaN,정환
3,네. 근데 내일 바로 가신대요.,야!,정환
4,NaN,"손으로 드럽게, 씨..",정환
...,...,...,...
3703,NaN,우리 엄마 외롭지 않게 해주셔서 감사합니다 아저씨,선우
3704,NaN,쌍문동에서 오신 치타 사모님이 조용히 계시면 식이 진행되도록 하겠습니다.,동룡
3705,야 사회 내가 본다고 그랬잖아,사천에서 겨우 올라오 주제에 동룡이 잘 봐,택
3706,NaN,신랑 입장!,동룡


In [ ]:
df.isnull().sum()

Q       1106
A        140
role       0
dtype: int64

In [ ]:
!pip install transformers
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
q_nan_idx = df[df['Q'].isnull()].index

In [ ]:
# q가 nan인 qa의 a
q_nan_a = []
for i in q_nan_idx:
  q_nan_a.append(df['A'][i])
#df['A'][q_nan_idx]
q_nan_a

In [ ]:
total = []

for i in range(len(df)):
  if (not pd.isna(df['Q'][i])):
    total.append(df['Q'][i])
  if (not pd.isna(df['A'][i])):
    total.append(df['A'][i])

In [ ]:
embeddings = model.encode(total, convert_to_tensor=True)

In [ ]:
k = 2

for i in range(len(q_nan_a)):
  dialog = []

  query = q_nan_a[i]
  query_embedding = model.encode(query, convert_to_tensor=True)
  cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
  cos_scores = cos_scores.cpu()

  top_results = np.argpartition(-cos_scores, range(k))[0:k]

  for idx in top_results[0:k]:
    dialog.append(total[idx].strip())

  df['Q'][q_nan_idx[i]] = dialog[1]

In [ ]:
df.isnull().sum()

Q         0
A       140
role      0
dtype: int64

In [ ]:
a_nan_idx = df[df['A'].isnull()].index

In [ ]:
# a가 nan인 qa의 q
a_nan_q = []
for i in a_nan_idx:
  a_nan_q.append(df['Q'][i])


In [ ]:
k = 2

for i in range(len(a_nan_q)):
  dialog = []

  query = a_nan_q[i]
  query_embedding = model.encode(query, convert_to_tensor=True)
  cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
  cos_scores = cos_scores.cpu()

  top_results = np.argpartition(-cos_scores, range(k))[0:k]

  for idx in top_results[0:k]:
    dialog.append(total[idx].strip())

  df['A'][a_nan_idx[i]] = dialog[1]

In [ ]:
df.isnull().sum()

Q       0
A       0
role    0
dtype: int64

In [ ]:
len(df)

3708

## q나 a가 공백 (' ', '') 인 데이터 삭제


In [ ]:
df.drop(df[(df['Q'] == ' ') | (df['A'] == '')].index,inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Q,A,role
0,정환아. 한 마디 해라.,"아, 진짜 이런 것 좀 하지 마세요.",정환
1,"♪ 실례, 실례하세요.",고개 숙이며) 안녕하세요..!!,정환
2,♪ 맞아! 맞아! 맞-아! 어떻게 알고 왔죠.,너 어떻게 알고 왔냐?,정환
3,네. 근데 내일 바로 가신대요.,야!,정환
4,"아휴, 씨..!!","손으로 드럽게, 씨..",정환
...,...,...,...
3447,"엄마, 아저씨 참 좋은 분 같아.",우리 엄마 외롭지 않게 해주셔서 감사합니다 아저씨,선우
3448,혹시 그대는 쌍문동에서 말로만 듣던 치타 사모님?,쌍문동에서 오신 치타 사모님이 조용히 계시면 식이 진행되도록 하겠습니다.,동룡
3449,야 사회 내가 본다고 그랬잖아,사천에서 겨우 올라오 주제에 동룡이 잘 봐,택
3450,결혼해.,신랑 입장!,동룡


In [ ]:
df[df['A'] == ' ']

,Q,A,role
2476,,,선우


In [ ]:
df.drop(2529,inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Q,A,role
0,정환아. 한 마디 해라.,"아, 진짜 이런 것 좀 하지 마세요.",정환
1,"♪ 실례, 실례하세요.",고개 숙이며) 안녕하세요..!!,정환
2,♪ 맞아! 맞아! 맞-아! 어떻게 알고 왔죠.,너 어떻게 알고 왔냐?,정환
3,네. 근데 내일 바로 가신대요.,야!,정환
4,"아휴, 씨..!!","손으로 드럽게, 씨..",정환
...,...,...,...
3446,"엄마, 아저씨 참 좋은 분 같아.",우리 엄마 외롭지 않게 해주셔서 감사합니다 아저씨,선우
3447,혹시 그대는 쌍문동에서 말로만 듣던 치타 사모님?,쌍문동에서 오신 치타 사모님이 조용히 계시면 식이 진행되도록 하겠습니다.,동룡
3448,야 사회 내가 본다고 그랬잖아,사천에서 겨우 올라오 주제에 동룡이 잘 봐,택
3449,결혼해.,신랑 입장!,동룡


## 데이터 증강에 필요한 함수

In [ ]:
def sentence_shuffle(sentence):

  if (sentence == '' or sentence == ' ') :
    return ''
    
  new_sentences = []

  if (len(sentence.split()) == 1):

    new_sentences.append("")

  elif (len(sentence.split()) == 2):

    new_sentences.append(sentence.split()[1] + " " + sentence.replace(sentence.split()[1],""))

  elif (len(sentence.split()) == 3):

    new_sentences.append(sentence.split()[1] + " " + sentence.replace(sentence.split()[1],""))

    new_sentences.append(sentence.split()[2] + " " + sentence.replace(sentence.split()[2],""))

  else :

    new_sentences.append(sentence.split()[1] + " " + sentence.replace(sentence.split()[1],""))

    new_sentences.append(sentence.split()[2] + " " + sentence.replace(sentence.split()[2],""))

    new_sentences.append(sentence.split()[len(sentence.split())-1] + " " + sentence.replace(sentence.split()[len(sentence.split())-1],""))
    
  new_sentence = new_sentences[randint(0, len(new_sentences)-1)]

  return new_sentence

In [ ]:
def word_shuffle(dialog):
  words = re.split("[ ]",dialog)
  while '' in words:    
    words.remove('')
  
  i = randint(0, len(words)-2)
  words[i], words[i+1]=words[i+1], words[i]

  result = ""
  for s in words:
      result += s + " "
  return result.strip()

In [ ]:
def word_remove(dialog):
  file_path = '/content/drive/Shareddrives/22-2 NLP 프로젝트/지호/stopwords.txt'

  with open(file_path) as f:
      lines = f.readlines()
  stopwords = [line.rstrip('\n') for line in lines]

  words = re.split("[ ]",dialog)
  while '' in words:    
      words.remove('')
  tmp = set(words)&set(stopwords)

  i = randint(0, len(tmp)-1)

  for word in words:
    if word == list(tmp)[i]:
      words.remove(word)
      break

  result = ""
  for s in words:
      result += s + " "
  return result.strip()

In [ ]:
from random import *

In [ ]:
def similar(query):
  dialog=[]

  k = 4
  
  query_embedding = model.encode(query, convert_to_tensor=True)
  cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
  cos_scores = cos_scores.cpu()

  top_results = np.argpartition(-cos_scores, range(k))[0:k]

  for idx in top_results[0:k]:
    dialog.append(total[idx].strip())

  return [dialog[1],dialog[2],dialog[3]] # list로 리턴

In [ ]:
def aug_q (q):
  extra = similar(q)                 ## 유사도 높은 3가지 Q 
  extra.append(sentence_shuffle(q))  ## 문장순서 랜덤 Q

  try:
    ss= word_shuffle(q)       ## 단어 2 셔플 Q
    extra.append(ss)
  except:
    extra.append(q)

  try:
    dd = word_remove(q)       ## 불용어제거 Q
    extra.append(dd)
  except :
    extra.append('')

  extra.append(q)                    ## origin Q

  return extra

def aug_a (a):
  extra = []
  extra.append(sentence_shuffle(a)) ## 문장순서 랜덤 A
  
  try:
    ss= word_shuffle(a)      ## 단어 2 셔플 A
    extra.append(ss)
  except:
    extra.append(a)

  try:
    dd = word_remove(a)      ## 단어 1 제거 A
    extra.append(dd)
  except:
    extra.append('')

  extra.append(a)                   ## origin A

  return extra

## 7 x 4 가짓수 만들기

In [ ]:
ex_q = df['Q'][12]
ex_a = df['A'][12]
ex_role = df['role'][12]

extra_q = aug_q(ex_q)
extra_a = aug_a(ex_a)

In [ ]:
extra_q

['선우가?', '근데 선우는?', '근데 선우는?', '선우는!? 우리 ', '선우는!? 우리', '선우는!?', '우리 선우는!?']

In [ ]:
extra_a

['와요.  뒤에  ', '와요. 뒤에', '', ' 뒤에 와요. ']

In [ ]:
total_q = []
total_a = []
total_role = []

for q in extra_q :
  for a in extra_a:
    total_q.append(q)
    total_a.append(a)
    total_role.append(ex_role)

sample = {'Q':total_q, 'A':total_a, 'role':total_role}
sam = pd.DataFrame(sample)
sam

,Q,A,role
0,선우가?,와요. 뒤에,정환
1,선우가?,와요. 뒤에,정환
2,선우가?,,정환
3,선우가?,뒤에 와요.,정환
4,근데 선우는?,와요. 뒤에,정환
5,근데 선우는?,와요. 뒤에,정환
6,근데 선우는?,,정환
7,근데 선우는?,뒤에 와요.,정환
8,근데 선우는?,와요. 뒤에,정환
9,근데 선우는?,와요. 뒤에,정환


In [ ]:
df

,Q,A,role
0,정환아. 한 마디 해라.,"아, 진짜 이런 것 좀 하지 마세요.",정환
1,"♪ 실례, 실례하세요.",고개 숙이며) 안녕하세요..!!,정환
2,♪ 맞아! 맞아! 맞-아! 어떻게 알고 왔죠.,너 어떻게 알고 왔냐?,정환
3,네. 근데 내일 바로 가신대요.,야!,정환
4,"아휴, 씨..!!","손으로 드럽게, 씨..",정환
...,...,...,...
3446,"엄마, 아저씨 참 좋은 분 같아.",우리 엄마 외롭지 않게 해주셔서 감사합니다 아저씨,선우
3447,혹시 그대는 쌍문동에서 말로만 듣던 치타 사모님?,쌍문동에서 오신 치타 사모님이 조용히 계시면 식이 진행되도록 하겠습니다.,동룡
3448,야 사회 내가 본다고 그랬잖아,사천에서 겨우 올라오 주제에 동룡이 잘 봐,택
3449,결혼해.,신랑 입장!,동룡


In [ ]:
total_q = []
total_a = []
total_role = []

for i in range(len(df)):
  ex_q = df['Q'][i]
  ex_a = df['A'][i]
  ex_role = df['role'][i]

  extra_q = aug_q(ex_q)
  extra_a = aug_a(ex_a)


  for q in extra_q :
    for a in extra_a:
      total_q.append(q)
      total_a.append(a)
      total_role.append(ex_role)


sample = {'Q':total_q, 'A':total_a, 'role':total_role}
sam = pd.DataFrame(sample)
sam

,Q,A,role
0,정환아..너한테 할 말 있어서..,"진짜 아, 이런 것 좀 하지 마세요.",정환
1,정환아..너한테 할 말 있어서..,"아, 이런 진짜 것 좀 하지 마세요.",정환
2,정환아..너한테 할 말 있어서..,"아, 진짜 것 좀 하지 마세요.",정환
3,정환아..너한테 할 말 있어서..,"아, 진짜 이런 것 좀 하지 마세요.",정환
4,야. 정환아.,"진짜 아, 이런 것 좀 하지 마세요.",정환
...,...,...,...
96623,,네 동네 친구들이에요. 잘 부탁드립니다!,선우
96624,친구들?,부탁드립니다! 네 동네 친구들이에요. 잘,선우
96625,친구들?,네 동네 친구들이에요. 부탁드립니다! 잘,선우
96626,친구들?,동네 친구들이에요. 잘 부탁드립니다!,선우


In [ ]:
sam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96628 entries, 0 to 96627
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       96628 non-null  object
 1   A       96628 non-null  object
 2   role    96628 non-null  object
dtypes: object(3)
memory usage: 2.2+ MB


In [ ]:
len(sam[sam['A'] == ''])

19831

In [ ]:
sam.to_csv('/content/drive/MyDrive/kubig_nlp프로젝트/10 31/final_augmented.csv',encoding='utf-8')

In [ ]:
sam.head()

,Q,A,role
0,정환아..너한테 할 말 있어서..,"진짜 아, 이런 것 좀 하지 마세요.",정환
1,정환아..너한테 할 말 있어서..,"아, 이런 진짜 것 좀 하지 마세요.",정환
2,정환아..너한테 할 말 있어서..,"아, 진짜 것 좀 하지 마세요.",정환
3,정환아..너한테 할 말 있어서..,"아, 진짜 이런 것 좀 하지 마세요.",정환
4,야. 정환아.,"진짜 아, 이런 것 좀 하지 마세요.",정환


In [ ]:
sam.loc[1100:1150]

,Q,A,role
1100,"야, 됐지?",닫잖아!! 야. 브라질 떡볶이 문 닫겠다.토요일에 일찍,동룡
1101,"야, 됐지?",야. 떡볶이 브라질 문 닫겠다.토요일에 일찍 닫잖아!!,동룡
1102,"야, 됐지?",,동룡
1103,"야, 됐지?",야. 브라질 떡볶이 문 닫겠다.토요일에 일찍 닫잖아!!,동룡
1104,,닫잖아!! 야. 브라질 떡볶이 문 닫겠다.토요일에 일찍,동룡
1105,,야. 떡볶이 브라질 문 닫겠다.토요일에 일찍 닫잖아!!,동룡
1106,,,동룡
1107,,야. 브라질 떡볶이 문 닫겠다.토요일에 일찍 닫잖아!!,동룡
1108,됐거든?,닫잖아!! 야. 브라질 떡볶이 문 닫겠다.토요일에 일찍,동룡
1109,됐거든?,야. 떡볶이 브라질 문 닫겠다.토요일에 일찍 닫잖아!!,동룡
